In [1]:
!pip install -q huggingface_hub
!pip install -q -U trl transformers accelerate peft
!pip install -q -U datasets bitsandbytes einops wandb
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/transformers.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/peft.git to /scratch/local/14526190/pip-req-build-or8lfpzs
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /scratch/local/14526190/pip-req-build-or8lfpzs
  Resolved https://github.com/huggingface/peft.git to commit cfe35a7878b44e017836f0b0c0c3b3e9e0cb738b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers.git to /scratch/local/14526190/pip-req-build-4vcoolev
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /scratch/local/14526190/pip-req-build-4vcoolev
  Resolved https://github.com/huggingface/transformers.git to commit cc3e4781854a52cf090ffde28d884a527dab67

In [8]:
from huggingface_hub import notebook_login
notebook_login()


In [1]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
import torch.distributed as dist
import os

# Set the master address to localhost and the master port to a free port
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12355'  # Choose an open port
os.environ["NCCL_SOCKET_IFNAME"] = 'eth0'

# if not dist.is_initialized():
    # Initialize the process group
    dist.init_process_group(
        backend='nccl',  # Assuming NCCL backend for GPU operations
        init_method='env://',
        world_size=1,  # The number of processes in the distributed group
        rank=0  # The rank of the current process (starts at 0)
    )

# Your code to create tensors and perform operations goes here

# For example, creating a tensor and performing all_reduce
tensor = torch.randn(10).cuda()
dist.all_reduce(tensor)

# Don't forget to clean up
dist.destroy_process_group()


In [2]:
relative_path_to_data = '../data/ukey/train.json'

dataset = load_dataset('json', data_files={'train': relative_path_to_data}, split="train")
base_model_name = "meta-llama/Llama-2-7b-hf"


In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

In [4]:
device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True
)
base_model.config.use_cache = False

/home/sosa.s/.local/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/sosa.s/.local/lib/python3.10/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [5]:
base_model.config.pretraining_tp = 1 

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [7]:
output_dir = "./results"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    max_steps=500
)

max_seq_length = 512

trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [8]:
%env NCCL_DEBUG=INFO

env: NCCL_DEBUG=INFO


In [9]:
trainer.train()

import os
output_dir = os.path.join(output_dir, "final_checkpoint")
trainer.model.save_pretrained(output_dir)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sebastiansosa. Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


c1106a-s29:2168420:2168420 [0] NCCL INFO cudaDriverVersion 12020

c1106a-s29:2168420:2168420 [0] bootstrap.cc:40 NCCL WARN Bootstrap : no socket interface found
c1106a-s29:2168420:2168420 [0] NCCL INFO init.cc:87 -> 3
c1106a-s29:2168420:2168420 [0] NCCL INFO init.cc:105 -> 3
c1106a-s29:2168420:2168420 [0] NCCL INFO init.cc:1220 -> 3


RuntimeError: NCCL Error 3: internal error

In [ ]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map=device_map, torch_dtype=torch.bfloat16)
text = "..."
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))